# Classificatore Cani e Gatti

Il modello è una conversione da TensorFlow a PyTorch di un classificatore equivalente

### Preparazione ambiente

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)
if device.type == "cuda":
    print("Nome dispositivo:", torch.cuda.get_device_name(0))

Usando dispositivo: cuda
Nome dispositivo: Tesla T4


In [4]:
# Verifica installazione PyTorch e ambiente
import sys
import torchvision
print("Python executable:", sys.executable)
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)

# Test device detection
print("CUDA disponible:", torch.cuda.is_available())
print("Device selezionato:", device)

# Info su compatibilità AMD
print("\n=== Info AMD GPU ===")
print("Supporto ROCm su Windows: Limitato/Non ufficiale")
print("GPU AMD RX 7700 XT: Riconosciuta dal sistema ma PyTorch usa CPU")
print("Per accelerazione GPU AMD: ROCm su Linux o DirectML (sperimentale)")

# Test basic operations
test_tensor = torch.randn(3, 3)
print(f"\nTest tensor: {test_tensor.shape} su device: {test_tensor.device}")
print("PyTorch installato e funzionante! ✅")

Python executable: /usr/bin/python3
PyTorch version: 2.8.0+cu126
Torchvision version: 0.23.0+cu126
CUDA disponible: True
Device selezionato: cuda

=== Info AMD GPU ===
Supporto ROCm su Windows: Limitato/Non ufficiale
GPU AMD RX 7700 XT: Riconosciuta dal sistema ma PyTorch usa CPU
Per accelerazione GPU AMD: ROCm su Linux o DirectML (sperimentale)

Test tensor: torch.Size([3, 3]) su device: cpu
PyTorch installato e funzionante! ✅


### Preparazione dei dati

Import del dataset da internet

In [5]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor()
])

test_set = datasets.ImageFolder(root='data/catdogs/validation', transform=transform)

test_loader = DataLoader(test_set, batch_size=64, pin_memory=True, num_workers=8)


FileNotFoundError: [Errno 2] No such file or directory: 'data/catdogs/validation'

In [ ]:
import matplotlib.pyplot as plt

train_set = datasets.ImageFolder(root='data/catdogs/train', transform=transform)
train_loader = DataLoader(train_set, batch_size=128, shuffle=True, pin_memory=True, num_workers=8)

images, labels = next(iter(train_loader))

fig, axes = plt.subplots(1, 5, figsize=(15, 3))
for i in range(5):
    img = images[i].permute(1, 2, 0)  # CHW --> HWC
    label = labels[i].item()
    axes[i].imshow(img)
    axes[i].set_title(f"Label: {label}")

plt.tight_layout()
plt.show()

Data augmentation

In [ ]:
data_aug = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.RandomRotation(40, fill=128),
    transforms.RandomAffine(
        degrees=0,
        translate=(0.2, 0.2),
        scale=(0.8, 1.2),
        shear=(-5.5, 5.5),
        fill=128
    ),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

train_set.transform = data_aug

In [ ]:
images, labels = next(iter(train_loader))

fig, axes = plt.subplots(1, 5, figsize=(15, 3))
for i in range(5):
    img = images[i].permute(1, 2, 0)  # CHW --> HWC
    label = labels[i].item()
    axes[i].imshow(img)
    axes[i].set_title(f"Label: {label}")

plt.tight_layout()
plt.show()

### Preparazione CNN

Rete Neurale

In [ ]:
class PetsCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_stack = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc_stack = nn.Sequential(
            nn.Flatten(),
            # output_size = (input_size - kernel_size) / stride + 1
            # da fare per tutti i layer conv e pool, alla fine viene 64 canali * 17 * 17
            nn.Linear(64 * 17 * 17, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_stack(x)
        x = self.fc_stack(x)
        return x

cnn = PetsCNN().to(device)

Funzione di costo e ottimizzatore

In [ ]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(cnn.parameters(), lr=0.001)

### Training

In [ ]:
epochs = 50

train_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(epochs):
    cnn.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device, non_blocking=True), data[1].to(device, non_blocking=True)

        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = cnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        predicted = (outputs > 0.5).long()
        total_train += labels.size(0)
        correct_train += (predicted == labels.long()).sum().item()

    avg_loss = running_loss / len(train_loader)
    train_acc = 100 * correct_train / total_train
    train_losses.append(avg_loss)
    train_accuracies.append(train_acc)

    cnn.eval()
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = cnn(images)
            predicted = (outputs > 0.5).long().squeeze()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_acc = 100 * correct_val / total_val
    val_accuracies.append(val_acc)

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {avg_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%')

print('Training Terminato')

### Test

In [ ]:
cnn.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = cnn(images)
        predicted = (outputs > 0.5).long().squeeze()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuratezza finale sul test set: {accuracy:.2%}')

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot Loss
ax1.plot(range(1, epochs + 1), train_losses, 'b-', label='Training Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss nel tempo')
ax1.legend()
ax1.grid(True)

# Plot Accuracy
ax2.plot(range(1, epochs + 1), train_accuracies, 'b-', label='Training Accuracy')
ax2.plot(range(1, epochs + 1), val_accuracies, 'r-', label='Validation Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Training vs Validation Accuracy')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

# Stampa risultati finali
print(f"\nRisultati finali:")
print(f"Training Accuracy: {train_accuracies[-1]:.2f}%")
print(f"Validation Accuracy: {val_accuracies[-1]:.2f}%")